In [1]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import tiktoken

In [2]:
GPT_CONFIG_124M = {
    'vocab_size': 50257, 
    'context_length': 1024,
    'emb_dim': 768,
    'n_heads': 12, 
    'n_layers': 12,
    'drop_rate': 0.1,
    'qkv_bias': False
}

In [3]:
class DummyTransformerBlock(nn.Module):
    def __init__(self, cnf):
        super().__init__()
    
    def forward(self, x):
        return x
    
class DummyLayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()
    
    def forward(self, x):
        return x


class DummyGPTModel(nn.Module):
    def __init__(self, cnf):
        super().__init__()
        self.tok_emb = nn.Embedding(cnf['vocab_size'], cnf['emb_dim'])
        self.pos_emb = nn.Embedding(cnf['context_length'], cnf['emb_dim'])
        self.drop_emb = nn.Dropout(cnf['drop_rate'])
        
        self.trf_blocks = nn.Sequential(*[DummyTransformerBlock(cnf) for _ in range(cnf['n_layers'])])
        
        self.final_norm = DummyLayerNorm(cnf['emb_dim'])
        self.out_head = nn.Linear(cnf['emb_dim'], cnf['vocab_size'], bias=False)
    
    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits     

In [4]:
txt1 = "Every effort moves you"
txt2 = "Every day holds a"

tokenizer = tiktoken.get_encoding('gpt2')
batch = []
batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch)
print(batch.shape)
print(batch)

torch.Size([2, 4])
tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


In [ ]:
model = DummyGPTModel(GPT_CONFIG_124M)
logits = model(batch)